In [1]:
import os
import comet_ml
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CometLogger

comet_ml.init(api_key="j1ToDZ4nYZRCvxL2k9brOVhxk",
              project_name="comet-pytorch-lightning-practice")
# arguments made to CometLogger are passed on to the comet_ml.Experiment class
comet_logger = CometLogger()

COMET INFO: Valid Comet API Key saved in /Users/victoriaono/.comet.config (set COMET_CONFIG to change where it is saved).
CometLogger will be initialized in online mode


In [3]:
import torch
import torch.nn.functional as F
import pytorch_lightning as pl

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader


class Model(pl.LightningModule):
    def __init__(self, layer_size=784):
        super().__init__()
        self.save_hyperparameters()
        self.l1 = torch.nn.Linear(layer_size, 10)

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self(x), y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_nb):
        x, y = batch
        y_hat = self.forward(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [4]:
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 256 if AVAIL_GPUS else 64

In [5]:
# Init our model
model = Model()

# Init DataLoader from MNIST Dataset
train_ds = MNIST(
    PATH_DATASETS, train=True, download=True, transform=transforms.ToTensor()
)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

eval_ds = MNIST(
    PATH_DATASETS, train=False, download=True, transform=transforms.ToTensor()
)
eval_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)

comet_logger.log_hyperparams({"batch_size": BATCH_SIZE})

# Initialize a trainer
trainer = Trainer(gpus=AVAIL_GPUS, max_epochs=3, logger=comet_logger)

# Train the model ⚡
trainer.fit(model, train_loader, eval_loader)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/victoriaono/comet-pytorch-lightning-practice/e050b40ae08e4619bdf41ac647b6bbaa

/Users/victoriaono/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:478: LightningDeprecationWarning: Setting `Trainer(gpus=0)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=0)` instead.
  rank_zero_deprecation(
COMET INFO: Couldn't find a Git repository in '/Users/victoriaono/CAMELS' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type   | Params
--------------------------

Sanity Checking: 0it [00:00, ?it/s]

/Users/victoriaono/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/victoriaono/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/victoriaono/comet-pytorch-lightning-practice/e050b40ae08e4619bdf41ac647b6bbaa
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     loss [282]      : (1.2172653675079346, 2.293651819229126)
COMET INFO:     train_loss [56] : (1.2503927946090698, 1.8651692867279053)
COMET INFO:     val_loss [3]    : (1.561282753944397, 1.6066087484359741)
COMET INFO:   Parameters:
COMET INFO:     batch_size : 64
COMET INFO:     layer_size : 784
COMET INFO:   Uploads:
COMET INFO:     conda-environment-definition : 1
COMET INFO:     conda-info                   : 1
COMET INFO:     conda-specificatio